In [ ]:
# Célula 1: APENAS instalação
!pip install transformers datasets accelerate evaluate -U
print("--- Bibliotecas atualizadas ---")

In [ ]:
# Célula 2: APENAS reiniciar
import os
print("Forçando o reinício do ambiente para carregar as bibliotecas novas...")
os.kill(os.getpid(), 9)

In [ ]:
# Célula 3: Imports (SÓ DEPOIS DO REINÍCIO)
import pandas as pd
import numpy as np
from google.colab import drive
import os
import evaluate
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

print("Bibliotecas novas importadas com sucesso.")

In [ ]:
# Célula 4: Carregar Dados
print("Montando Google Drive...")
drive.mount('/content/drive')

drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/Goodreads/'
stratified_file_path = os.path.join(drive_base_path, 'goodreads_sample_200k_stratified.csv')

print(f"Carregando amostra de {stratified_file_path}...")
df_sample = pd.read_csv(stratified_file_path)
df_sample = df_sample.dropna(subset=['review_text', 'sentiment']).copy()

print(f"Sucesso! Amostra de {len(df_sample)} linhas carregada.")

In [ ]:
# Célula 5: Pré-processamento e Tokenização

# 1. Mapeamento de Labels
label_map = {'Negativo': 0, 'Neutro': 1, 'Positivo': 2}
df_sample['label'] = df_sample['sentiment'].map(label_map)

# 2. Carregar Tokenizador
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 3. Divisão Treino/Teste
train_df, test_df = train_test_split(
    df_sample,
    test_size=0.2,
    random_state=42,
    stratify=df_sample['label']
)

# 4. Converter para formato Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 5. Função de Tokenização
def tokenize_function(examples):
    return tokenizer(
        examples['review_text'],
        padding='max_length',
        truncation=True,
        max_length=256
    )

# Aplicar Tokenização
print("Iniciando tokenização...")
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Limpar e Formatar
cols_to_remove = [
    'user_id', 'book_id', 'review_id', 'rating', 'review_text', 'date_added',
    'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments', 'sentiment', '__index_level_0__'
]
tokenized_train_dataset = tokenized_train_dataset.remove_columns(cols_to_remove)
tokenized_test_dataset = tokenized_test_dataset.remove_columns(cols_to_remove)
tokenized_train_dataset.set_format('torch')
tokenized_test_dataset.set_format('torch')

print("Tokenização concluída e datasets formatados.")

In [ ]:
# Célula 6: Carregar Modelo e Definir Métricas

# num_labels=3 informa ao modelo que esperamos 3 saídas (Negativo, Neutro, Positivo)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

metric = evaluate.load("f1")

# Função que calcula as métricas durante a avaliação
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    f1_result = metric.compute(predictions=predictions, references=labels, average="macro")
    return {"f1_macro": f1_result["f1"]}

print("Modelo e função de métrica prontos.")

In [ ]:
# Célula 7: Configurar os Argumentos de Treinamento

output_dir_path = os.path.join(drive_base_path, 'BERT_Checkpoints_Roberta_Novo')
print(f"Os checkpoints serão salvos em: {output_dir_path}")

steps_per_epoch = 159965 // 16
print(f"Cálculo: 1 época = aproximadamente {steps_per_epoch} passos.")

training_args = TrainingArguments(
    output_dir=output_dir_path,

    # --- Parâmetros de Treinamento ---
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,

    # --- Parâmetros de Avaliação e Salvamento ---
    eval_steps=steps_per_epoch,       # Avaliar a cada X passos (equiv. 1 época)
    save_steps=steps_per_epoch,       # Salvar a cada X passos (equiv. 1 época)
    save_total_limit=3,               # Salva todos os 3 checkpoints (1 por época)

    logging_dir=os.path.join(output_dir_path, 'logs'),
    logging_steps=500,
    report_to="none"
)

print("\nTrainingArguments (modo de compatibilidade) configurado com sucesso.")

In [ ]:
# Célula 8: Inicializar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
)
print("Trainer inicializado. Pronto para o treinamento.")

In [ ]:
# Célula 9: Iniciar o Treinamento
print("Iniciando o treinamento do modelo BERT/RoBERTa...")
trainer.train()
print("Treinamento concluído.")

In [ ]:
# CÉLULA 10 - Avaliar o Checkpoint da Época 1

print("--- Avaliando Checkpoint 1 (checkpoint-9997) ---")

checkpoint_epoca_1 = os.path.join(output_dir_path, "checkpoint-9997")

model_epoca_1 = AutoModelForSequenceClassification.from_pretrained(checkpoint_epoca_1)

trainer_epoca_1 = Trainer(
    model=model_epoca_1,
    args=training_args,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics
)

resultados_epoca_1 = trainer_epoca_1.evaluate()
print(resultados_epoca_1)

In [ ]:
# CÉLULA 11 - Avaliar o Checkpoint da Época 2

print("--- Avaliando Checkpoint 2 (checkpoint-19994) ---")

checkpoint_epoca_2 = os.path.join(output_dir_path, "checkpoint-19994")

model_epoca_2 = AutoModelForSequenceClassification.from_pretrained(checkpoint_epoca_2)

trainer_epoca_2 = Trainer(
    model=model_epoca_2,
    args=training_args,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics
)

resultados_epoca_2 = trainer_epoca_2.evaluate()
print(resultados_epoca_2)

### 3. Análise Comparativa e Demonstração Final

Com os resultados da Época 2 (F1-Macro: 0.718) como nosso melhor desempenho do BERT, podemos compará-los visualmente com os modelos da Etapa 2.

In [ ]:
# Célula 12: Gráfico Comparativo - F1-Score Macro
import matplotlib.pyplot as plt
import seaborn as sns

modelos = ['Naive Bayes (Baseline)', 'Regressão Logística (Principal)', 'BERT/RoBERTa (SOTA)']
f1_macros = [0.34, 0.57, resultados_epoca_2['eval_f1_macro']]

plt.figure(figsize=(10, 6))
barplot = sns.barplot(x=modelos, y=f1_macros, palette='viridis')

plt.title('Comparação de Desempenho dos Modelos (F1-Score Macro)', fontsize=16)
plt.ylabel('F1-Score Macro', fontsize=12)
plt.ylim(0, 1.0)

for p in barplot.patches:
    barplot.annotate(format(p.get_height(), '.3f'),
                       (p.get_x() + p.get_width() / 2., p.get_height()),
                       ha = 'center', va = 'center',
                       xytext = (0, 9),
                       textcoords = 'offset points')

plt.show()

In [ ]:
# Célula 13: Gerar Matriz de Confusão do BERT (Época 2)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

print("Gerando predições do modelo BERT (Época 2) para a Matriz de Confusão...")

predicoes = trainer_epoca_2.predict(tokenized_test_dataset)

y_pred_bert = np.argmax(predicoes.predictions, axis=1)

y_true_bert = predicoes.label_ids

labels_nomes = ['Negativo', 'Neutro', 'Positivo']

cm_bert = confusion_matrix(y_true_bert, y_pred_bert, labels=[0, 1, 2])

print("Matriz de Confusão - BERT/RoBERTa (Principal)")
disp = ConfusionMatrixDisplay(confusion_matrix=cm_bert, display_labels=labels_nomes)
fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title('Matriz de Confusão - BERT/RoBERTa (Época 2)', fontsize=16)
plt.xlabel('Predito', fontsize=12)
plt.ylabel('Verdadeiro', fontsize=12)
plt.show()

In [ ]:
# Célula 14: Gerar Imagem do Relatório de Classificação (BERT)
from transformers import pipeline
import shutil
from sklearn.metrics import classification_report

print("Gerando Relatório de Classificação do BERT (Época 2)...")

report = classification_report(
    y_true_bert,
    y_pred_bert,
    target_names=labels_nomes,
    output_dict=True
)

report_df = pd.DataFrame(report).transpose()
report_df = report_df.round(2)

report_df.loc['accuracy', 'support'] = report_df.loc['accuracy', 'f1-score']
report_df.loc['accuracy', ['precision', 'recall', 'f1-score']] = np.nan

report_df['support'] = report_df['support'].astype(int)

print("Relatório como DataFrame:")
display(report_df)

fig, ax = plt.subplots(figsize=(8, 4))
ax.axis('off')
ax.axis('tight')

plot_df = report_df.round(2).fillna('')

table = ax.table(
    cellText=plot_df.values,
    colLabels=plot_df.columns,
    rowLabels=plot_df.index,
    loc='center',
    cellLoc='center'
)

table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1.2, 1.2)

plt.title('Relatório de Classificação - BERT/RoBERTa (Época 2)', fontsize=16, y=0.75)

plt.savefig('bert_classification_report.png', bbox_inches='tight', dpi=150)
print("\nImagem do relatório salva como 'bert_classification_report.png'")

plt.show()

### 4. Demonstração do Modelo (Inferência)

Esta célula demonstra o "print de recomendação" — como o modelo final classifica novas reviews.

In [ ]:
# Célula 15: Pipeline de Demonstração

print("Carregando pipeline de análise de sentimento com o *melhor* modelo (Época 2)...")

id2label = {0: 'Negativo', 1: 'Neutro', 2: 'Positivo'}

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model_epoca_2,
    tokenizer=tokenizer,
    device=0
)

sentiment_pipeline.model.config.id2label = id2label


print("\n--- TESTANDO O MODELO --- (Inferência em reviews novas)")

# --- Teste 1: Review Positiva ---
review_positiva = "This is the best book I have ever read. The story was incredible and the characters were perfect. I loved it!"
resultado_pos = sentiment_pipeline(review_positiva)
print(f"Review: '{review_positiva}'")
print(f"Predição do Modelo: {resultado_pos}")


# --- Teste 2: Review Negativa ---
review_negativa = "I hated this book. It was boring, the plot made no sense, and the ending was terrible. A complete waste of time."
resultado_neg = sentiment_pipeline(review_negativa)
print(f"\nReview: '{review_negativa}'")
print(f"Predição do Modelo: {resultado_neg}")


# --- Teste 3: Review Neutra/Ambígua ---
review_neutra = "The book was okay. The beginning was slow, but it got better near the end. I'm not sure if I would recommend it."
resultado_neu = sentiment_pipeline(review_neutra)
print(f"\nReview: '{review_neutra}'")
print(f"Predição do Modelo: {resultado_neu}")